# יישום מערכת מומחה לחיות

דוגמה מתוך [AI for Beginners Curriculum](http://github.com/microsoft/ai-for-beginners).

בדוגמה זו, ניישם מערכת פשוטה מבוססת ידע כדי לזהות חיה על סמך כמה מאפיינים פיזיים. ניתן לייצג את המערכת באמצעות עץ AND-OR הבא (זהו חלק מהעץ המלא, ניתן להוסיף בקלות עוד חוקים):

![](../../../../translated_images/AND-OR-Tree.5592d2c70187f283703c8e9c0d69d6a786eb370f4ace67f9a7aae5ada3d260b0.he.png)


## מערכת מומחה משלנו עם הסקה לאחור

בואו ננסה להגדיר שפה פשוטה לייצוג ידע המבוססת על כללי ייצור. נשתמש במחלקות של Python כמילות מפתח להגדרת כללים. יהיו למעשה 3 סוגים של מחלקות:
* `Ask` מייצגת שאלה שצריך לשאול את המשתמש. היא מכילה את קבוצת התשובות האפשריות.
* `If` מייצגת כלל, והיא רק תחביר נוח לאחסון תוכן הכלל.
* `AND`/`OR` הן מחלקות שמייצגות ענפי AND/OR בעץ. הן פשוט מאחסנות את רשימת הפרמטרים בתוכן. כדי לפשט את הקוד, כל הפונקציונליות מוגדרת במחלקת האב `Content`.


In [1]:
class Ask():
    def __init__(self,choices=['y','n']):
        self.choices = choices
    def ask(self):
        if max([len(x) for x in self.choices])>1:
            for i,x in enumerate(self.choices):
                print("{0}. {1}".format(i,x),flush=True)
            x = int(input())
            return self.choices[x]
        else:
            print("/".join(self.choices),flush=True)
            return input()

class Content():
    def __init__(self,x):
        self.x=x
        
class If(Content):
    pass

class AND(Content):
    pass

class OR(Content):
    pass

במערכת שלנו, זיכרון העבודה יכיל את רשימת **העובדות** כ**זוגות תכונה-ערך**. בסיס הידע יכול להיות מוגדר כמילון גדול שממפה פעולות (עובדות חדשות שצריכות להיות מוכנסות לזיכרון העבודה) לתנאים, המובעים כביטויי AND-OR. בנוסף, ניתן גם `לשאול` כמה עובדות.


In [2]:
rules = {
    'default': Ask(['y','n']),
    'color' : Ask(['red-brown','black and white','other']),
    'pattern' : Ask(['dark stripes','dark spots']),
    'mammal': If(OR(['hair','gives milk'])),
    'carnivor': If(OR([AND(['sharp teeth','claws','forward-looking eyes']),'eats meat'])),
    'ungulate': If(['mammal',OR(['has hooves','chews cud'])]),
    'bird': If(OR(['feathers',AND(['flies','lies eggs'])])),
    'animal:monkey' : If(['mammal','carnivor','color:red-brown','pattern:dark spots']),
    'animal:tiger' : If(['mammal','carnivor','color:red-brown','pattern:dark stripes']),
    'animal:giraffe' : If(['ungulate','long neck','long legs','pattern:dark spots']),
    'animal:zebra' : If(['ungulate','pattern:dark stripes']),
    'animal:ostrich' : If(['bird','long nech','color:black and white','cannot fly']),
    'animal:pinguin' : If(['bird','swims','color:black and white','cannot fly']),
    'animal:albatross' : If(['bird','flies well'])
}

כדי לבצע הסקה לאחור, נגדיר את המחלקה `Knowledgebase`. היא תכיל:
* `זיכרון` פעיל - מילון שממפה תכונות לערכים
* `חוקי` בסיס הידע בפורמט שהוגדר לעיל

שתי השיטות המרכזיות הן:
* `get` להשגת הערך של תכונה, תוך ביצוע הסקה במידת הצורך. לדוגמה, `get('color')` ישיג את הערך של משבצת הצבע (הוא ישאל אם צריך, וישמור את הערך לשימוש עתידי בזיכרון הפעיל). אם נשאל `get('color:blue')`, הוא ישאל על צבע, ואז יחזיר ערך `y`/`n` בהתאם לצבע.
* `eval` מבצע את ההסקה בפועל, כלומר עובר על עץ AND/OR, מעריך תתי-מטרות, וכדומה.


In [3]:
class KnowledgeBase():
    def __init__(self,rules):
        self.rules = rules
        self.memory = {}
        
    def get(self,name):
        if ':' in name:
            k,v = name.split(':')
            vv = self.get(k)
            return 'y' if v==vv else 'n'
        if name in self.memory.keys():
            return self.memory[name]
        for fld in self.rules.keys():
            if fld==name or fld.startswith(name+":"):
                # print(" + proving {}".format(fld))
                value = 'y' if fld==name else fld.split(':')[1]
                res = self.eval(self.rules[fld],field=name)
                if res!='y' and res!='n' and value=='y':
                    self.memory[name] = res
                    return res
                if res=='y':
                    self.memory[name] = value
                    return value
        # field is not found, using default
        res = self.eval(self.rules['default'],field=name)
        self.memory[name]=res
        return res
                
    def eval(self,expr,field=None):
        # print(" + eval {}".format(expr))
        if isinstance(expr,Ask):
            print(field)
            return expr.ask()
        elif isinstance(expr,If):
            return self.eval(expr.x)
        elif isinstance(expr,AND) or isinstance(expr,list):
            expr = expr.x if isinstance(expr,AND) else expr
            for x in expr:
                if self.eval(x)=='n':
                    return 'n'
            return 'y'
        elif isinstance(expr,OR):
            for x in expr.x:
                if self.eval(x)=='y':
                    return 'y'
            return 'n'
        elif isinstance(expr,str):
            return self.get(expr)
        else:
            print("Unknown expr: {}".format(expr))

עכשיו בואו נגדיר את בסיס הידע שלנו על בעלי חיים ונבצע את הייעוץ. שימו לב שהשיחה הזו תציג לכם שאלות. תוכלו לענות על ידי הקלדת `y`/`n` לשאלות כן-לא, או על ידי ציון מספר (0..N) לשאלות עם תשובות מרובות אפשרויות ארוכות יותר.


In [4]:
kb = KnowledgeBase(rules)
kb.get('animal')

hair
y/n
sharp teeth
y/n
claws
y/n
forward-looking eyes
y/n
color
0. red-brown
1. black and white
2. other
has hooves
y/n
long neck
y/n
long legs
y/n
pattern
0. dark stripes
1. dark spots


'giraffe'

## שימוש ב-PyKnow להסקה קדימה

בדוגמה הבאה, ננסה ליישם הסקה קדימה באמצעות אחת הספריות לייצוג ידע, [PyKnow](https://github.com/buguroo/pyknow/). **PyKnow** היא ספרייה ליצירת מערכות הסקה קדימה ב-Python, אשר עוצבה להיות דומה למערכת הקלאסית הישנה [CLIPS](http://www.clipsrules.net/index.html).

יכולנו גם ליישם בעצמנו שרשור קדימה ללא בעיות רבות, אך מימושים נאיביים בדרך כלל אינם יעילים במיוחד. לצורך התאמת חוקים יעילה יותר, נעשה שימוש באלגוריתם מיוחד בשם [Rete](https://en.wikipedia.org/wiki/Rete_algorithm).


In [5]:
import sys
!{sys.executable} -m pip install git+https://github.com/buguroo/pyknow/

  Cloning https://github.com/buguroo/pyknow/ to /tmp/pip-req-build-3cqeulyl
  Running command git clone --filter=blob:none --quiet https://github.com/buguroo/pyknow/ /tmp/pip-req-build-3cqeulyl
  Resolved https://github.com/buguroo/pyknow/ to commit 48818336f2e9a126f1964f2d8dc22d37ff800fe8
  Preparing metadata (setup.py) ... done
  Using cached frozendict-1.2.tar.gz (2.6 kB)
  Preparing metadata (setup.py) ... done
  Using cached schema-0.6.7-py2.py3-none-any.whl (14 kB)
  Created wheel for pyknow: filename=pyknow-1.7.0-py3-none-any.whl size=34228 sha256=b7de5b09292c4007667c72f69b98d5a1b5f7324ff15f9dd8e077c3d5f7aade42
  Stored in directory: /tmp/pip-ephem-wheel-cache-k7jpave7/wheels/81/1a/d3/f6c15dbe1955598a37755215f2a10449e7418500d7bd4b9508
  Created wheel for frozendict: filename=frozendict-1.2-py3-none-any.whl size=3148 sha256=2863d55c240d2409cddf05ccfe600591f8478681549fc97555c47c90dc6bb160
  Stored in directory: /home/rg/.cache/pip/wheels/49/ac/f8/cb8120244e710bdb479c86198b03c7b08c

In [13]:
from pyknow import *
#import pyknow

אנו נגדיר את המערכת שלנו ככיתה שיורשת את `KnowledgeEngine`. כל כלל מוגדר על ידי פונקציה נפרדת עם סימון `@Rule`, שמציין מתי הכלל צריך לפעול. בתוך הכלל, אנו יכולים להוסיף עובדות חדשות באמצעות פונקציית `declare`, והוספת העובדות הללו תגרום לכך שכללים נוספים יופעלו על ידי מנוע ההסקה קדימה.


In [14]:
class Animals(KnowledgeEngine):
    @Rule(OR(
           AND(Fact('sharp teeth'),Fact('claws'),Fact('forward looking eyes')),
           Fact('eats meat')))
    def cornivor(self):
        self.declare(Fact('carnivor'))
        
    @Rule(OR(Fact('hair'),Fact('gives milk')))
    def mammal(self):
        self.declare(Fact('mammal'))

    @Rule(Fact('mammal'),
          OR(Fact('has hooves'),Fact('chews cud')))
    def hooves(self):
        self.declare('ungulate')
        
    @Rule(OR(Fact('feathers'),AND(Fact('flies'),Fact('lays eggs'))))
    def bird(self):
        self.declare('bird')
        
    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark spots'))
    def monkey(self):
        self.declare(Fact(animal='monkey'))

    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark stripes'))
    def tiger(self):
        self.declare(Fact(animal='tiger'))

    @Rule(Fact('ungulate'),
          Fact('long neck'),
          Fact('long legs'),
          Fact(pattern='dark spots'))
    def giraffe(self):
        self.declare(Fact(animal='giraffe'))

    @Rule(Fact('ungulate'),
          Fact(pattern='dark stripes'))
    def zebra(self):
        self.declare(Fact(animal='zebra'))

    @Rule(Fact('bird'),
          Fact('long neck'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def straus(self):
        self.declare(Fact(animal='ostrich'))

    @Rule(Fact('bird'),
          Fact('swims'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def pinguin(self):
        self.declare(Fact(animal='pinguin'))

    @Rule(Fact('bird'),
          Fact('flies well'))
    def albatros(self):
        self.declare(Fact(animal='albatross'))
        
    @Rule(Fact(animal=MATCH.a))
    def print_result(self,a):
          print('Animal is {}'.format(a))
                    
    def factz(self,l):
        for x in l:
            self.declare(x)

לאחר שהגדרנו בסיס ידע, אנו ממלאים את הזיכרון הפעיל שלנו בעובדות ראשוניות, ואז קוראים לשיטת `run()` כדי לבצע את ההסקה. ניתן לראות כתוצאה מכך שעובדות חדשות שהוסקו מתווספות לזיכרון הפעיל, כולל העובדה הסופית לגבי החיה (אם הגדרנו את כל העובדות הראשוניות בצורה נכונה).


In [15]:
ex1 = Animals()
ex1.reset()
ex1.factz([
    Fact(color='red-brown'),
    Fact(pattern='dark stripes'),
    Fact('sharp teeth'),
    Fact('claws'),
    Fact('forward looking eyes'),
    Fact('gives milk')])
ex1.run()
ex1.facts

Animal is tiger


FactList([(0, InitialFact()),
          (1, Fact(color='red-brown')),
          (2, Fact(pattern='dark stripes')),
          (3, Fact('sharp teeth')),
          (4, Fact('claws')),
          (5, Fact('forward looking eyes')),
          (6, Fact('gives milk')),
          (7, Fact('mammal')),
          (8, Fact('carnivor')),
          (9, Fact(animal='tiger'))])


---

**כתב ויתור**:  
מסמך זה תורגם באמצעות שירות תרגום מבוסס בינה מלאכותית [Co-op Translator](https://github.com/Azure/co-op-translator). בעוד שאנו שואפים לדיוק, יש לקחת בחשבון שתרגומים אוטומטיים עשויים להכיל שגיאות או אי דיוקים. המסמך המקורי בשפתו המקורית צריך להיחשב כמקור סמכותי. עבור מידע קריטי, מומלץ להשתמש בתרגום מקצועי על ידי אדם. איננו נושאים באחריות לאי הבנות או לפרשנויות שגויות הנובעות משימוש בתרגום זה.
